# Feedback Equivalence

The purpose of this demo is to demonstrate that, with a fully actuated system and a good enough system model, we can cancel all the dynamics and make our system behave however we want.

In [ ]:
import time
import numpy as np
import mujoco

In [2]:

xml = """
<mujoco>
  <option gravity = "0 0 -9.81" integrator="RK4" timestep="0.005"/>
   <worldbody>
      <light diffuse="0 0 0" pos="0 0 10" dir="0 0 -1"/>
      <body pos="0 0 0" euler="0 180 0">
         <joint name="joint0" type="hinge" axis = "0 -1 0" pos = "0 0 0.5"/>
         <geom type="cylinder" size="0.05 0.5" rgba="1 0 0 1" mass="1"/>
         <body pos="0 -0.0 -1" euler="0 0 0">
           <joint name="joint1" type="hinge" axis = "0 -1 0" pos = "0 0 0.5"/>
           <geom type="cylinder" size="0.05 0.5" rgba="0 1 0 1" mass="1"/>
         </body>
      </body>
    </worldbody>
   <actuator>
     <motor name="torque0" joint="joint0"  gear="1" />
     <motor name="torque1" joint="joint1"  gear="1" />
   </actuator>
</mujoco>
"""
model = mujoco.MjModel.from_xml_string(xml)

## Controller

This controller does one of two things. If given a reference point in state space $q_{\mathrm{ref}}$, it will move the system to that point. If $q_{\mathrm{ref}}$ is an empty list, it will cancel the dynamics and force the system to act as a single pendulum instead of a double pendulum. In general, we can change the parameters of our imposed physics however we want.

In [15]:
def controller(model, data):
    # this controller is called by mujoco during the control loop
    kp = 10
    kd = 2 * np.sqrt(kp)
    M = np.zeros((2,2))
    # Get mass matrix and coriolis + gravity from mujoco
    mujoco.mj_fullM(model, M, data.qM)
    C = data.qfrc_bias

    if q_ref !=[]:
        # go to set point
        th_dd = -kp * (data.qpos - q_ref) - kd * data.qvel
    else:
        # emulate the phsyics of a single pendulum
        # try playing around with the parameters if you are curious:
        g = 9.81
        l = 0.5
        b = 0.1
        th_dd = [g / l * np.sin(data.qpos[0]) - b * data.qvel[0], - kp * data.qpos[1] - kd * data.qvel[1]]

    # This controller cancels the dynamics and imposes whatever physics we want
    tau = C + np.matmul(M, th_dd)
    data.ctrl[0] = tau[0]
    data.ctrl[1] = tau[1]

## Mujoco simulation

This constructs our mujoco simulation using the double pendulum model and outputs a video.

In [18]:
# Load the model and data
data = mujoco.MjData(model)
mujoco.mj_resetDataKeyframe(model, data, 1)
data.qpos[0] = -3*np.pi/2
data.qpos[1] = 0.0
q_ref = [-4*np.pi/2, 0.5]
q_ref = []

mujoco.set_mjcb_control(controller)
DURATION  = 5   # seconds
FRAMERATE = 1/model.opt.timestep  # Hz

with mujoco.viewer.launch_passive(model, data) as viewer:
    viewer.cam.azimuth = 90
    viewer.cam.elevation = 5
    viewer.cam.distance =  8
    viewer.cam.lookat = np.array([0.0 , 0.0 , 0.0])
    while data.time < DURATION and viewer.is_running():
        step_start = time.time()
        first_time = time.time()

        # mj_step can be replaced with code that also evaluates
        # a policy and applies a control signal before stepping the physics.
        mujoco.mj_step(model, data)
        # input("wait")
        time.sleep(model.opt.timestep)
        
        # print(time.time() - start_time)
        # Pick up changes to the physics state, apply perturbations, update options from GUI.
        viewer.sync()